In this notebook, I implement the Continuous Bag of Words algorithm from scratch, which is a word2vec algorithm for calculating word embeddings. First, we must start off with some actual words data to train the CBOW model on. For this, I will use Text8, a text file containing the first 100 MB of cleaned text from Wikipedia. 

In CBOW, we predict a target word based on context words.

In [2]:
import gensim.downloader as api

# Load the text8 dataset
text8_dataset = api.load('text8')

# Convert it into a list of words
text8_words = [word for words in text8_dataset for word in words]

print(text8_words[:100])  # Print the first 100 words to check
print("Number of words:", len(text8_words))

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing']
17005207


Since in CBOW we are trying to predict the center word based on context words, we first need to define which context words correlate to each word. We do this by choosing a window size. Let's use a window size of 5. The below code creates a dictionary where each key is a word and its corresponding object is the context words.

In [3]:
words_to_context = {}
WINDOW_SIZE = 5

for index, word in enumerate(text8_words):
    # Calculate start and end indices for context
    start_index = max(0, index - WINDOW_SIZE)
    end_index = min(len(text8_words), index + WINDOW_SIZE + 1)

    # Get context words excluding the target word itself
    word_context = text8_words[start_index:index] + text8_words[index + 1:end_index]

    # Append context words to the list for this word in the dictionary
    if word in words_to_context:
        words_to_context[word].append(word_context)
    else:
        words_to_context[word] = [word_context]

Now that we have the context-target pairs ready, we need to encode these words into one-hot vectors so that they can be converted into word embeddings by training the parameters of the model to make a prediction of the target word given the context word. The code below one-hot encodes our words.

In [28]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# One-hot encode vocab
one_hot_encoder = OneHotEncoder() 
unique_text8_words = np.array(list(set(text8_words))).reshape(-1, 1)
one_hot_encoder.fit(unique_text8_words)
one_hot_encoder.transform(unique_text8_words)
unique_text8_words

array([['georgy'],
       ['mwanza'],
       ['nerezi'],
       ...,
       ['duveen'],
       ['lecardo'],
       ['kolchan']], dtype='<U100')

In [8]:
# Convert words in words_to_context dictionary to one-hot
word_to_one_hot = {word[0]: one_hot_encoder.transform([[word[0]]])
                   for word in unique_text8_words}
one_hot_words_to_contexts = {}
for word, contexts in words_to_context.items():
    # Transform each context word to one-hot using the map
    one_hot_contexts = [word_to_one_hot[context_word] for context_word in contexts if context_word in word_to_one_hot]
    # Assign the one-hot encoded context words to the one-hot encoded target word
    one_hot_words_to_contexts[word_to_one_hot[word]] = one_hot_contexts

TypeError: unhashable type: 'list'

In [15]:
print('test')

test
